In [2]:
import pandas as pd
from pathlib import Path

In [3]:
RAW = Path("data/raw")
OUT = Path("data/interim")
OUT.mkdir(parents=True, exist_ok=True)

In [5]:
data = pd.read_csv('/home/rzby/ffmc_dc/data/raw/export-ME48-2015-01-01-2024-01-01.csv', low_memory=False)
data.head()

,WMO ID,DATA TIMESTAMP,EVAPORATION 24HOURS MM EEE,RAINFALL 24H RRRR,TEMP DRYBULB C TTTTTT,WIND SPEED FF,RELATIVE HUMIDITY PC,PRESSURE QFF MB DERIVED
0,96595,2015-01-01 00:00:00.0 +0:00,7.0,7.6,23.4,0.0,98.3,1012.0
1,96595,2015-01-01 01:00:00.0 +0:00,NaN,NaN,24.6,0.0,92.8,1012.2
2,96595,2015-01-01 02:00:00.0 +0:00,NaN,NaN,27.0,0.0,80.8,1012.0
3,96595,2015-01-01 03:00:00.0 +0:00,NaN,NaN,29.8,0.0,71.2,1011.0
4,96595,2015-01-01 04:00:00.0 +0:00,NaN,NaN,30.7,0.0,65.4,1011.0


In [6]:
data['WMO ID'].unique()

array([96595, 96645, 96651, 96653, 96655])

In [7]:
data['WMO ID'] = data['WMO ID'].astype(str)